In [ ]:
import pandas as pd 
import numpy as np
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)